# Procesamiento de climatologías

Este _notebook_ contiene programas en Python para el procesamiento de climatologías. Una climatología se define como un conjunto de medidas estadísticas que resumen series temporales de variables climáticas como precipitación, temperatura, radiación solar y velocidad del viento, entre otras.

**Bibliotecas**

In [1]:
import os

import rasterio

from shapely.geometry import Point

import pandas as pd
import geopandas as gpd

## Entrada

In [2]:
# Directorio de datos de entrada
# INPUT_DATA_DIR = "C:/Users/mfvargas/raster/raster_recortados/"
INPUT_DATA_DIR = "/home/mfvargas/raster/raster_recortados/"

# Directorio de datos de salida
# OUTPUT_DATA_DIR = "C:/Users/mfvargas/raster/raster_recortados/"
OUTPUT_DATA_DIR = "/home/mfvargas/raster/salida/"

# Shapefile de salida
OUTPUT_SHAPEFILE = "CHELSA_1981.shp"

# Archivo CSV de salida
OUTPUT_CSV = "CHELSA_1981.csv"

# Valor de NO DATA
NO_DATA_VALUE = 65535

## Procesamiento

In [3]:
# DataFrame
centros_gdf = gpd.GeoDataFrame(columns=['geometry', 'x', 'y'], crs={"init": "epsg:4326"})

# Contador de archivos
f = 0

# Recorrido de los archivos del directorio de entrada
for file in os.listdir(INPUT_DATA_DIR):
    anio = file[12:16]
    mes = file[17:19]
    
    # Condiciones
    #if mes != '12':
    #    continue
    
    if anio != '1981':
        continue
    
    print("Año:", anio, "Mes:", mes)
    centros_gdf['c_' + anio + '_' + mes] = 65535
    
    dataset = rasterio.open(INPUT_DATA_DIR + "/" + file)
    print("Archivo:", file, "Ancho:", dataset.width, "Largo:", dataset.height)
    
    # Recuperación de los valores de la banda 1 en un arreglo
    band = dataset.read(1)
    
    # Recorrido de las filas y columnas de la banda
    i = 0
    t = 0
    for row in band:
        j = 0
        for value in row:
            if value != NO_DATA_VALUE:
                coordinates = dataset.transform * (j, i)
                x = coordinates[0]
                y = coordinates[1]
                point = Point(coordinates)
                # print(t, i, j, x, y, point, value)
                if (f == 0):
                    centros_gdf.loc[t] = [point] + [x] + [y] + [value]
                else:
                    centros_gdf.loc[[t], ['c_' + anio + '_' + mes]] = value
                t += 1
            j += 1
            
        i += 1
    f += 1

# Columna de la suma
centros_gdf['suma'] = centros_gdf.iloc[:,3:].sum(axis=1)    
    
# Columna del promedio
centros_gdf['promedio'] = centros_gdf.iloc[:,3:].mean(axis=1)

/home/mfvargas/anaconda3/envs/imn3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Año: 1981 Mes: 01
Archivo: CHELSA_prec_1981_01_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 04
Archivo: CHELSA_prec_1981_04_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 11
Archivo: CHELSA_prec_1981_11_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 09
Archivo: CHELSA_prec_1981_09_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 05
Archivo: CHELSA_prec_1981_05_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 02
Archivo: CHELSA_prec_1981_02_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 03
Archivo: CHELSA_prec_1981_03_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 07
Archivo: CHELSA_prec_1981_07_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 10
Archivo: CHELSA_prec_1981_10_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 08
Archivo: CHELSA_prec_1981_08_V1.2.1_recortado.tiff Ancho: 411 Largo: 382
Año: 1981 Mes: 06
Archivo: CHELSA_prec_1981_06_V1.2.1_recortado.tiff Ancho: 411 Largo: 382

## Salida

In [ ]:
centros_gdf

In [ ]:
centros_gdf.to_file(OUTPUT_DATA_DIR + "/" + OUTPUT_SHAPEFILE)
centros_gdf.to_file(OUTPUT_DATA_DIR + "/" + OUTPUT_CSV, driver="CSV")